In [1]:
import pandas as pd
import os
from sklearn.metrics import classification_report
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

In [2]:
import sys
sys.path.append('../src')

from models.classification_methods import get_classification_report

In [3]:
test_results_path = '../reports/test_results/'

list_df_t = os.listdir(test_results_path)
list_df_t.sort()
list_df_t

['DummyClassifier_bo_top_mentioned_timelines_Texts_test_results.csv',
 'DummyClassifier_bo_users_Stance_test_results.csv',
 'DummyClassifier_bo_users_Timeline_test_results.csv',
 'DummyClassifier_cl_top_mentioned_timelines_Texts_test_results.csv',
 'DummyClassifier_cl_users_Stance_test_results.csv',
 'DummyClassifier_cl_users_Timeline_test_results.csv',
 'DummyClassifier_co_top_mentioned_timelines_Texts_test_results.csv',
 'DummyClassifier_co_users_Stance_test_results.csv',
 'DummyClassifier_co_users_Timeline_test_results.csv',
 'DummyClassifier_gl_top_mentioned_timelines_Texts_test_results.csv',
 'DummyClassifier_gl_users_Stance_test_results.csv',
 'DummyClassifier_gl_users_Timeline_test_results.csv',
 'DummyClassifier_ig_top_mentioned_timelines_Texts_test_results.csv',
 'DummyClassifier_ig_users_Stance_test_results.csv',
 'DummyClassifier_ig_users_Timeline_test_results.csv',
 'DummyClassifier_lu_top_mentioned_timelines_Texts_test_results.csv',
 'DummyClassifier_lu_users_Stance_test_r

In [4]:
# Target list
target_list = [
    'ig',
    'bo', 
    'cl', 
    'co', 
    'gl', 
    'lu'
]

dict_cp = {
    'cl':'Hydrox.',
    'lu':'Lula',
    'co':'Sinovac',
    'ig':'Church',
    'gl':'Globo TV',
    'bo':'Bolsonaro',
}

names = list(dict_cp.values())
names

['Hydrox.', 'Lula', 'Sinovac', 'Church', 'Globo TV', 'Bolsonaro']

## Create complete table

In [5]:
# (vectorizer,estimator, path_sring) 
results_tuples_stance = [
    # Stance
    ("Stance", "-" ,"dummy", "DummyClassifier_{target}_users_Stance_test_results.csv"),
    ("Stance", "tf-idf" ,"xgb", "XGBClassifier_TfidfVectorizer_{target}_users_Stance_test_results.csv"),
    ("Stance", "bertabaporu-base" ,"xgb", "bertimbau_xgb_{target}_users_emb_Stance_test_results.csv"),
    ("Stance", "-" ,"bertabaporu-base", "bert_classifier_pablocosta_bertabaporu_base_uncased_{target}_Stance_test_results.csv"),
    ("Stance", "-",  "llama3:7b zero-shot", "llama3_{target}_Stance_prompt2_Stance_test_results.csv"),
    
    # Texts
    ("Texts", "-" ,"dummy", "DummyClassifier_{target}_top_mentioned_timelines_Texts_test_results.csv"),
    ("Texts", "tf-idf" ,"xgb", "XGBClassifier_TfidfVectorizer_{target}_top_mentioned_timelines_Texts_test_results.csv"),
    ("Texts", "bertabaporu-base" ,"xgb", "bertimbau_xgb_{target}_top_mentioned_timelines_emb_Texts_test_results.csv"),
    ("Texts", "-", "llama3:7b zero-shot [5]", "llama3_{target}_filtered_Texts5_prompt2_Texts_test_results.csv"),
    ("Texts", "-", "llama3:7b zero-shot [10]", "llama3_{target}_filtered_Texts10_prompt2_Texts_test_results.csv"),
    # ("Texts", "-", "llama3:7b zero-shot [20]", "llama3_{target}_filtered_Texts20_prompt2_Texts_test_results.csv"),
    # ("Texts", "-", "llama3:7b zero-shot [30]", "llama3_{target}_filtered_Texts30_prompt2_Texts_test_results.csv"),
    # ("Texts", "-", "llama3:7b zero-shot [40]", "llama3_{target}_filtered_Texts40_prompt2_Texts_test_results.csv"),
    
    # Timeline
    ("Timeline", "-" ,"dummy", "DummyClassifier_{target}_users_Timeline_test_results.csv"),
    ("Timeline", "tf-idf" ,"xgb", "XGBClassifier_TfidfVectorizer_{target}_users_Timeline_test_results.csv"),
    ("Timeline", "bertabaporu-base" ,"xgb", "bertimbau_xgb_{target}_users_emb_Timeline_test_results.csv"),
    ("Timeline", "-", "llama3:7b zero-shot [5]", "llama3_{target}_filteredTimeline5_prompt2_Timeline_test_results.csv"),
    # ("Timeline", "-", "llama3:7b zero-shot [10]", "llama3_{target}_filteredTimeline10_prompt2_Timeline_test_results.csv"),
    # ("Timeline", "-", "llama3:7b zero-shot [20]", "llama3_{target}_filteredTimeline20_prompt2_Timeline_test_results.csv"),
    # ("Timeline", "-", "llama3:7b zero-shot [30]", "llama3_{target}_filteredTimeline30_prompt2_Timeline_test_results.csv"),
    # ("Timeline", "-", "llama3:7b zero-shot [40]", "llama3_{target}_filteredTimeline40_prompt2_Timeline_test_results.csv"),
]

list_results = []
for text_col, vectorizer, estimator, path_results in results_tuples_stance:
    
    list_cr = []
    
    for target in target_list:
        
        
        path = test_results_path + path_results.format(target = target)
        df_results = pd.read_csv(path)
        df_results_or = df_results.copy()
        
        # get classification report df
        df_classification_report = get_classification_report(df_results.test, df_results.pred, cr_args = {})
        
        # create multindex
        column_indexes = [(metric,dict_cp[target]) for metric in df_classification_report.columns]
        multi_index_cols = pd.MultiIndex.from_tuples(column_indexes, names=['metric', 'target'])
        rows_indexes = [(text_col, vectorizer, estimator, cl) for cl in df_classification_report.index]
        multi_index_rows = pd.MultiIndex.from_tuples(rows_indexes, names=['text_col','vectorizer', 'estimator', 'class'])
        df_classification_report.columns = multi_index_cols
        df_classification_report.index = multi_index_rows
        
        # print(text_col, vectorizer, estimator,target)
        # print(path)
        # display(df_classification_report)
        
        list_cr.append(df_classification_report)
        
    df_results = pd.concat(list_cr, axis = 1)
    
    list_results.append(df_results)
    
df_results_final = pd.concat(list_results)

In [6]:
df_results_final

metric                                                   precision    recall  \
target                                                      Church    Church   
text_col vectorizer estimator               class                              
Stance   -          dummy                   against       0.565943  1.000000   
                                            accuracy      0.565943  0.565943   
                                            weighted avg  0.320292  0.565943   
                                            macro avg     0.282972  0.500000   
                                            for           0.000000  0.000000   
...                                                            ...       ...   
Timeline -          llama3:7b zero-shot [5] against       0.660606  0.643068   
                                            weighted avg  0.612677  0.611018   
                                            accuracy      0.611018  0.611018   
                                            macro avg     0.605396  0.606149   
                                            for           0.550186  0.569231   

metric                                                    f1-score  \
target                                                      Church   
text_col vectorizer estimator               class                    
Stance   -          dummy                   against       0.722814   
                                            accuracy      0.565943   
                                            weighted avg  0.409072   
                                            macro avg     0.361407   
                                            for           0.000000   
...                                                            ...   
Timeline -          llama3:7b zero-shot [5] against       0.651719   
                                            weighted avg  0.611711   
                                            accuracy      0.611018   
                                            macro avg     0.605633   
                                            for           0.559546   

metric                                                       support  \
target                                                        Church   
text_col vectorizer estimator               class                      
Stance   -          dummy                   against       339.000000   
                                            accuracy        0.565943   
                                            weighted avg  599.000000   
                                            macro avg     599.000000   
                                            for           260.000000   
...                                                              ...   
Timeline -          llama3:7b zero-shot [5] against       339.000000   
                                            weighted avg  599.000000   
                                            accuracy        0.611018   
                                            macro avg     599.000000   
                                            for           260.000000   

metric                                                   precision    recall  \
target                                                   Bolsonaro Bolsonaro   
text_col vectorizer estimator               class                              
Stance   -          dummy                   against       0.861702  1.000000   
                                            accuracy      0.861702  0.861702   
                                            weighted avg  0.742531  0.861702   
                                            macro avg     0.430851  0.500000   
                                            for           0.000000  0.000000   
...                                                            ...       ...   
Timeline -          llama3:7b zero-shot [5] against       0.921260  0.722222   
                                            weighted avg  0.830126  0.707447   
         

In [7]:
df_results_final.to_excel("../reports/table_complete_results.xlsx")

## Create table f1 macro

In [8]:
mask_f1 = [True if  "f1-score" in col else False for col in df_results_final.columns]
mask_macro = [True if  "macro avg" in col else False for col in df_results_final.index]

f1_macro_df = df_results_final.loc[mask_macro,mask_f1]
f1_macro_df[('f1-score','overall')] = f1_macro_df.mean(axis=1)

f1_macro_df

metric                                                       f1-score  \
target                                                         Church   
text_col vectorizer       estimator               class                 
Stance   -                dummy                   macro avg  0.361407   
         tf-idf           xgb                     macro avg  0.702021   
         bertabaporu-base xgb                     macro avg  0.853801   
         -                bertabaporu-base        macro avg  0.866621   
                          llama3:7b zero-shot     macro avg  0.729458   
Texts    -                dummy                   macro avg  0.361407   
         tf-idf           xgb                     macro avg  0.596115   
         bertabaporu-base xgb                     macro avg  0.594754   
         -                llama3:7b zero-shot [5] macro avg  0.499052   
Timeline -                dummy                   macro avg  0.361407   
         tf-idf           xgb                     macro avg  0.699122   
         bertabaporu-base xgb                     macro avg  0.661773   
         -                llama3:7b zero-shot [5] macro avg  0.605633   

metric                                                                 \
target                                                      Bolsonaro   
text_col vectorizer       estimator               class                 
Stance   -                dummy                   macro avg  0.462857   
         tf-idf           xgb                     macro avg  0.595312   
         bertabaporu-base xgb                     macro avg  0.625289   
         -                bertabaporu-base        macro avg  0.739125   
                          llama3:7b zero-shot     macro avg  0.462857   
Texts    -                dummy                   macro avg  0.462857   
         tf-idf           xgb                     macro avg  0.501220   
         bertabaporu-base xgb                     macro avg  0.495578   
         -                llama3:7b zero-shot [5] macro avg  0.296142   
Timeline -                dummy                   macro avg  0.462857   
         tf-idf           xgb                     macro avg  0.728220   
         bertabaporu-base xgb                     macro avg  0.692262   
         -                llama3:7b zero-shot [5] macro avg  0.588752   

metric                                                                 \
target                                                        Hydrox.   
text_col vectorizer       estimator               class                 
Stance   -                dummy                   macro avg  0.334878   
         tf-idf           xgb                     macro avg  0.734366   
         bertabaporu-base xgb                     macro avg  0.830948   
         -                bertabaporu-base        macro avg  0.844778   
                          llama3:7b zero-shot     macro avg  0.638889   
Texts    -                dummy                   macro avg  0.334878   
         tf-idf           xgb                     macro avg  0.603727   
         bertabaporu-base xgb                     macro avg  0.609524   
         -                llama3:7b zero-shot [5] macro avg  0.469796   
Timeline -                dummy                   macro avg  0.334878   
         tf-idf           xgb                     macro avg  0.898944   
         bertabaporu-base xgb                     macro avg  0.776979   
         -                llama3:7b zero-shot [5] macro avg  0.639513   

metric                                                                 \
target                                                        Sinovac   
text_col vectorizer       estimator               class                 
Stance   -                dummy                   macro avg  0.351759   
         tf-idf           xgb                     macro avg  0.752640   
         bertabaporu-base xgb                     macro avg  0.808036   
         -                bertabaporu-base        macro

### Table for docs

In [9]:
f1_report = f1_macro_df.copy()
f1_report

metric                                                       f1-score  \
target                                                         Church   
text_col vectorizer       estimator               class                 
Stance   -                dummy                   macro avg  0.361407   
         tf-idf           xgb                     macro avg  0.702021   
         bertabaporu-base xgb                     macro avg  0.853801   
         -                bertabaporu-base        macro avg  0.866621   
                          llama3:7b zero-shot     macro avg  0.729458   
Texts    -                dummy                   macro avg  0.361407   
         tf-idf           xgb                     macro avg  0.596115   
         bertabaporu-base xgb                     macro avg  0.594754   
         -                llama3:7b zero-shot [5] macro avg  0.499052   
Timeline -                dummy                   macro avg  0.361407   
         tf-idf           xgb                     macro avg  0.699122   
         bertabaporu-base xgb                     macro avg  0.661773   
         -                llama3:7b zero-shot [5] macro avg  0.605633   

metric                                                                 \
target                                                      Bolsonaro   
text_col vectorizer       estimator               class                 
Stance   -                dummy                   macro avg  0.462857   
         tf-idf           xgb                     macro avg  0.595312   
         bertabaporu-base xgb                     macro avg  0.625289   
         -                bertabaporu-base        macro avg  0.739125   
                          llama3:7b zero-shot     macro avg  0.462857   
Texts    -                dummy                   macro avg  0.462857   
         tf-idf           xgb                     macro avg  0.501220   
         bertabaporu-base xgb                     macro avg  0.495578   
         -                llama3:7b zero-shot [5] macro avg  0.296142   
Timeline -                dummy                   macro avg  0.462857   
         tf-idf           xgb                     macro avg  0.728220   
         bertabaporu-base xgb                     macro avg  0.692262   
         -                llama3:7b zero-shot [5] macro avg  0.588752   

metric                                                                 \
target                                                        Hydrox.   
text_col vectorizer       estimator               class                 
Stance   -                dummy                   macro avg  0.334878   
         tf-idf           xgb                     macro avg  0.734366   
         bertabaporu-base xgb                     macro avg  0.830948   
         -                bertabaporu-base        macro avg  0.844778   
                          llama3:7b zero-shot     macro avg  0.638889   
Texts    -                dummy                   macro avg  0.334878   
         tf-idf           xgb                     macro avg  0.603727   
         bertabaporu-base xgb                     macro avg  0.609524   
         -                llama3:7b zero-shot [5] macro avg  0.469796   
Timeline -                dummy                   macro avg  0.334878   
         tf-idf           xgb                     macro avg  0.898944   
         bertabaporu-base xgb                     macro avg  0.776979   
         -                llama3:7b zero-shot [5] macro avg  0.639513   

metric                                                                 \
target                                                        Sinovac   
text_col vectorizer       estimator               class                 
Stance   -                dummy                   macro avg  0.351759   
         tf-idf           xgb                     macro avg  0.752640   
         bertabaporu-base xgb                     macro avg  0.808036   
         -                bertabaporu-base        macro

In [10]:
f1_report.reset_index(drop=False, inplace=True)
f1_report

metric  text_col        vectorizer                estimator      class  \
target                                                                   
0         Stance                 -                    dummy  macro avg   
1         Stance            tf-idf                      xgb  macro avg   
2         Stance  bertabaporu-base                      xgb  macro avg   
3         Stance                 -         bertabaporu-base  macro avg   
4         Stance                 -      llama3:7b zero-shot  macro avg   
5          Texts                 -                    dummy  macro avg   
6          Texts            tf-idf                      xgb  macro avg   
7          Texts  bertabaporu-base                      xgb  macro avg   
8          Texts                 -  llama3:7b zero-shot [5]  macro avg   
9       Timeline                 -                    dummy  macro avg   
10      Timeline            tf-idf                      xgb  macro avg   
11      Timeline  bertabaporu-base                      xgb  macro avg   
12      Timeline                 -  llama3:7b zero-shot [5]  macro avg   

metric  f1-score                                                              
target    Church Bolsonaro   Hydrox.   Sinovac  Globo TV      Lula   overall  
0       0.361407  0.462857  0.334878  0.351759  0.372519  0.344578  0.371333  
1       0.702021  0.595312  0.734366  0.752640  0.665243  0.654412  0.683999  
2       0.853801  0.625289  0.830948  0.808036  0.781858  0.766993  0.777821  
3       0.866621  0.739125  0.844778  0.840499  0.862638  0.796338  0.825000  
4       0.729458  0.462857  0.638889  0.578779  0.770678  0.699381  0.646674  
5       0.361407  0.462857  0.334878  0.351759  0.372519  0.344578  0.371333  
6       0.596115  0.501220  0.603727  0.670805  0.545620  0.579427  0.582819  
7       0.594754  0.495578  0.609524  0.660542  0.534455  0.569597  0.577408  
8       0.499052  0.296142  0.469796  0.470381  0.490101  0.512954  0.456404  
9       0.361407  0.462857  0.334878  0.351759  0.372519  0.344578  0.371333  
10      0.699122  0.728220  0.898944  0.863813  0.598358  0.752710  0.756861  
11      0.661773  0.692262  0.776979  0.778471  0.585105  0.645518  0.690018  
12      0.605633  0.588752  0.639513  0.528509  0.560086  0.514680  0.572862

In [11]:
f1_report.columns

MultiIndex([(  'text_col',          ''),
            ('vectorizer',          ''),
            ( 'estimator',          ''),
            (     'class',          ''),
            (  'f1-score',    'Church'),
            (  'f1-score', 'Bolsonaro'),
            (  'f1-score',   'Hydrox.'),
            (  'f1-score',   'Sinovac'),
            (  'f1-score',  'Globo TV'),
            (  'f1-score',      'Lula'),
            (  'f1-score',   'overall')],
           names=['metric', 'target'])

In [12]:
new_columns = [col[0] if col[1] == '' else col[1] for col in f1_report.columns]
new_columns

['text_col',
 'vectorizer',
 'estimator',
 'class',
 'Church',
 'Bolsonaro',
 'Hydrox.',
 'Sinovac',
 'Globo TV',
 'Lula',
 'overall']

In [13]:
f1_report.columns = new_columns
f1_report

,text_col,vectorizer,estimator,class,Church,Bolsonaro,Hydrox.,Sinovac,Globo TV,Lula,overall
0,Stance,-,dummy,macro avg,0.361407,0.462857,0.334878,0.351759,0.372519,0.344578,0.371333
1,Stance,tf-idf,xgb,macro avg,0.702021,0.595312,0.734366,0.752640,0.665243,0.654412,0.683999
2,Stance,bertabaporu-base,xgb,macro avg,0.853801,0.625289,0.830948,0.808036,0.781858,0.766993,0.777821
3,Stance,-,bertabaporu-base,macro avg,0.866621,0.739125,0.844778,0.840499,0.862638,0.796338,0.825000
4,Stance,-,llama3:7b zero-shot,macro avg,0.729458,0.462857,0.638889,0.578779,0.770678,0.699381,0.646674
5,Texts,-,dummy,macro avg,0.361407,0.462857,0.334878,0.351759,0.372519,0.344578,0.371333
6,Texts,tf-idf,xgb,macro avg,0.596115,0.501220,0.603727,0.670805,0.545620,0.579427,0.582819
7,Texts,bertabaporu-base,xgb,macro avg,0.594754,0.495578,0.609524,0.660542,0.534455,0.569597,0.577408
8,Texts,-,llama3:7b zero-shot [5],macro avg,0.499052,0.296142,0.469796,0.470381,0.490101,0.512954,0.456404
9,Timeline,-,dummy,macro avg,0.361407,0.462857,0.334878,0.351759,0.372519,0.344578,0.371333


In [14]:
f1_report.drop(['class'],axis = 1, inplace=True)
f1_report

,text_col,vectorizer,estimator,Church,Bolsonaro,Hydrox.,Sinovac,Globo TV,Lula,overall
0,Stance,-,dummy,0.361407,0.462857,0.334878,0.351759,0.372519,0.344578,0.371333
1,Stance,tf-idf,xgb,0.702021,0.595312,0.734366,0.752640,0.665243,0.654412,0.683999
2,Stance,bertabaporu-base,xgb,0.853801,0.625289,0.830948,0.808036,0.781858,0.766993,0.777821
3,Stance,-,bertabaporu-base,0.866621,0.739125,0.844778,0.840499,0.862638,0.796338,0.825000
4,Stance,-,llama3:7b zero-shot,0.729458,0.462857,0.638889,0.578779,0.770678,0.699381,0.646674
5,Texts,-,dummy,0.361407,0.462857,0.334878,0.351759,0.372519,0.344578,0.371333
6,Texts,tf-idf,xgb,0.596115,0.501220,0.603727,0.670805,0.545620,0.579427,0.582819
7,Texts,bertabaporu-base,xgb,0.594754,0.495578,0.609524,0.660542,0.534455,0.569597,0.577408
8,Texts,-,llama3:7b zero-shot [5],0.499052,0.296142,0.469796,0.470381,0.490101,0.512954,0.456404
9,Timeline,-,dummy,0.361407,0.462857,0.334878,0.351759,0.372519,0.344578,0.371333


In [15]:
f1_report.insert(
    1, 
    "classifier", 
    f1_report.apply(
        lambda x: f"{x['vectorizer']} + {x['estimator']}" if x['vectorizer'] != '-' else x['estimator'],
        axis = 1
        ).to_list()

)
f1_report.drop(['estimator', 'vectorizer'],axis =1, inplace = True)
f1_report

,text_col,classifier,Church,Bolsonaro,Hydrox.,Sinovac,Globo TV,Lula,overall
0,Stance,dummy,0.361407,0.462857,0.334878,0.351759,0.372519,0.344578,0.371333
1,Stance,tf-idf + xgb,0.702021,0.595312,0.734366,0.752640,0.665243,0.654412,0.683999
2,Stance,bertabaporu-base + xgb,0.853801,0.625289,0.830948,0.808036,0.781858,0.766993,0.777821
3,Stance,bertabaporu-base,0.866621,0.739125,0.844778,0.840499,0.862638,0.796338,0.825000
4,Stance,llama3:7b zero-shot,0.729458,0.462857,0.638889,0.578779,0.770678,0.699381,0.646674
5,Texts,dummy,0.361407,0.462857,0.334878,0.351759,0.372519,0.344578,0.371333
6,Texts,tf-idf + xgb,0.596115,0.501220,0.603727,0.670805,0.545620,0.579427,0.582819
7,Texts,bertabaporu-base + xgb,0.594754,0.495578,0.609524,0.660542,0.534455,0.569597,0.577408
8,Texts,llama3:7b zero-shot [5],0.499052,0.296142,0.469796,0.470381,0.490101,0.512954,0.456404
9,Timeline,dummy,0.361407,0.462857,0.334878,0.351759,0.372519,0.344578,0.371333


In [16]:
f1_report.rename({"text_col":"input"}, axis = 1, inplace=True)
f1_report

,input,classifier,Church,Bolsonaro,Hydrox.,Sinovac,Globo TV,Lula,overall
0,Stance,dummy,0.361407,0.462857,0.334878,0.351759,0.372519,0.344578,0.371333
1,Stance,tf-idf + xgb,0.702021,0.595312,0.734366,0.752640,0.665243,0.654412,0.683999
2,Stance,bertabaporu-base + xgb,0.853801,0.625289,0.830948,0.808036,0.781858,0.766993,0.777821
3,Stance,bertabaporu-base,0.866621,0.739125,0.844778,0.840499,0.862638,0.796338,0.825000
4,Stance,llama3:7b zero-shot,0.729458,0.462857,0.638889,0.578779,0.770678,0.699381,0.646674
5,Texts,dummy,0.361407,0.462857,0.334878,0.351759,0.372519,0.344578,0.371333
6,Texts,tf-idf + xgb,0.596115,0.501220,0.603727,0.670805,0.545620,0.579427,0.582819
7,Texts,bertabaporu-base + xgb,0.594754,0.495578,0.609524,0.660542,0.534455,0.569597,0.577408
8,Texts,llama3:7b zero-shot [5],0.499052,0.296142,0.469796,0.470381,0.490101,0.512954,0.456404
9,Timeline,dummy,0.361407,0.462857,0.334878,0.351759,0.372519,0.344578,0.371333


In [17]:
#f1_report.set_index(['input'],inplace=True)
#f1_report.drop('input', axis = 1,inplace=True)

In [18]:
def generate_latex_with_multirow_and_bold(df):
    latex_code = ''
    latex_code += '\\begin{table}[H]'
    latex_code += "\\begin{tabular}{ll|rrrrrrr}\n\\toprule\n"
    latex_code += "input & classifier & Church & Bolsonaro & Hydrox. & Sinovac & Globo TV & Lula & overall \\\\ \n\\midrule\n"

    last_input = None
    multirow_count = 0

    for input_value in df['input'].unique():
        subset = df[df['input'] == input_value]
        max_overall_idx = subset['overall'].idxmax()

        for i, row in subset.iterrows():
            if row['input'] == last_input:
                latex_code += "& "
                multirow_count += 1
            else:
                if multirow_count > 0:
                    latex_code = latex_code.replace(f"multirow{{{multirow_count}}}", f"multirow{{{multirow_count + 1}}}", 1)
                if last_input is not None:
                    latex_code += "\\cmidrule(lr){1-9}\n"
                latex_code += f"\\multirow{{{1}}}{{*}}{{{row['input']}}} & "
                multirow_count = 1

            if i == max_overall_idx:
                row_data = [f"\\textbf{{{row[col]:.2f}}}" if col not in ['input', 'classifier'] else f"\\textbf{{{row[col]}}}" for col in df.columns[1:]]
                latex_code += " & ".join(row_data) + " \\\\ \n"
            else:
                latex_code += " & ".join([f"{row[col]:.2f}" if isinstance(row[col], float) else str(row[col]) for col in df.columns[1:]]) + " \\\\ \n"
            
            last_input = row['input']

    if multirow_count > 0:
        latex_code = latex_code.replace(f"multirow{{{multirow_count}}}", f"multirow{{{multirow_count + 1}}}", 1)

    latex_code += "\\cmidrule(lr){1-9}\n"
    latex_code += "\\bottomrule\n\\end{tabular}"
    latex_code += "\caption{F1 macro results}"
    latex_code += '\label{table:results_f1_macro}\n'
    latex_code += '\end{table}'

    return latex_code

In [19]:
f1_report.input = f1_report.input.map({
    "Stance": "S",
    "Timeline": 'UT',
    "Texts": 'UFT'
})

In [20]:
str_latex = generate_latex_with_multirow_and_bold(f1_report)

In [21]:
print(str_latex)

\begin{table}[H]\begin{tabular}{ll|rrrrrrr}
\toprule
input & classifier & Church & Bolsonaro & Hydrox. & Sinovac & Globo TV & Lula & overall \\ 
\midrule
\multirow{1}{*}{S} & dummy & 0.36 & 0.46 & 0.33 & 0.35 & 0.37 & 0.34 & 0.37 \\ 
& tf-idf + xgb & 0.70 & 0.60 & 0.73 & 0.75 & 0.67 & 0.65 & 0.68 \\ 
& bertabaporu-base + xgb & 0.85 & 0.63 & 0.83 & 0.81 & 0.78 & 0.77 & 0.78 \\ 
& \textbf{bertabaporu-base} & \textbf{0.87} & \textbf{0.74} & \textbf{0.84} & \textbf{0.84} & \textbf{0.86} & \textbf{0.80} & \textbf{0.82} \\ 
& llama3:7b zero-shot & 0.73 & 0.46 & 0.64 & 0.58 & 0.77 & 0.70 & 0.65 \\ 
\cmidrule(lr){1-9}
\multirow{1}{*}{UFT} & dummy & 0.36 & 0.46 & 0.33 & 0.35 & 0.37 & 0.34 & 0.37 \\ 
& \textbf{tf-idf + xgb} & \textbf{0.60} & \textbf{0.50} & \textbf{0.60} & \textbf{0.67} & \textbf{0.55} & \textbf{0.58} & \textbf{0.58} \\ 
& bertabaporu-base + xgb & 0.59 & 0.50 & 0.61 & 0.66 & 0.53 & 0.57 & 0.58 \\ 
& llama3:7b zero-shot [5] & 0.50 & 0.30 & 0.47 & 0.47 & 0.49 & 0.51 & 0.46 \\ 
\cm